In [1]:
import urllib.request
import re
import spacy
from spacy import displacy

nlp = spacy.load('en')

In [2]:
url1 = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exlimit=max&explaintext&titles=car&redirects="

x = urllib.request.urlopen(url1).read().decode('utf-8')

In [3]:
x

'{"batchcomplete":"","warnings":{"extracts":{"*":"\\"exlimit\\" was too large for a whole article extracts request, lowered to 1."}},"query":{"normalized":[{"from":"car","to":"Car"}],"pages":{"13673345":{"pageid":13673345,"ns":0,"title":"Car","extract":"A car (or automobile) is a wheeled motor vehicle used for transportation.  Most definitions of car say they run primarily on roads, seat one to eight people, have four tires, and mainly transport people rather than goods. Cars came into global use during the 20th century, and developed economies depend on them. The year 1886 is regarded as the birth year of the modern car when German inventor Karl Benz patented his Benz Patent-Motorwagen. Cars became widely available in the early 20th century. One of the first cars that were accessible to the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced animal-drawn carriages and carts, but took much longer 

In [4]:
sentence = '(\w[^.]*\.)'

sentences = re.findall(sentence,x)

sentences[3]
#cc_regex = re.compile(content_container)

'Cars came into global use during the 20th century, and developed economies depend on them.'

In [5]:
# Maybe need a cleanup like this, not sure yet
def preprocess_sentence(s):
    skip_chars = "\\="
    if not s:
        return ""
    elif s.startswith("\\"):
        return preprocess_sentence(s[1:])
    else:
        return s[0]+preprocess_sentence(s[1:])

print(sentences[25])
preprocess_sentence(sentences[25])


n\n\n== Etymology ==\nThe word car is believed to originate from the Latin word carrus or carrum (\"wheeled vehicle\"), or the Middle English word carre (meaning \"two-wheel cart\", from Old North French).


'nnn== Etymology ==nThe word car is believed to originate from the Latin word carrus or carrum ("wheeled vehicle"), or the Middle English word carre (meaning "two-wheel cart", from Old North French).'

In [6]:
# Save docs in a list
sentences_list = []
for i, doc in enumerate(nlp.pipe(sentences)):
    sentences_list.append(doc)
    

In [7]:
for w in sentences_list[50]:
    print(w, w.tag_, w.i)

nIn NNP 0
1807 CD 1
, , 2
Nic\u00e9phore NNP 3
Ni\u00e9pce JJ 4
and CC 5
his PRP$ 6
brother NN 7
Claude NNP 8
created VBD 9
what WP 10
was VBD 11
probably RB 12
the DT 13
world NN 14
's POS 15
first JJ 16
internal JJ 17
combustion NN 18
engine NN 19
( -LRB- 20
which WDT 21
they PRP 22
called VBD 23
a DT 24
Pyr\u00e9olophore NN 25
) -RRB- 26
, , 27
but CC 28
they PRP 29
chose VBD 30
to TO 31
install VB 32
it PRP 33
in IN 34
a DT 35
boat NN 36
on IN 37
the DT 38
river NN 39
Saone NN 40
in IN 41
France NNP 42
. . 43


In [8]:
# Test which tokens are options for being marked e1 and e2
for s in sentences_list[20:30]:
    print('SENTENCE: ', s,'\n\n')
    print("NOUN CANDIDATES:")
    for w in s:
        if(w.tag_ in ["NNS","NN"]):
            print(w)
        
    print("ENTITIES: ")
    for ent in s.ents:
        print(ent, ent.label_)
    print('######################################################\n\n')
    
    
'''
Useless entity types:
CARDINAL,
DATE
'''

SENTENCE:  The societal benefits include economic benefits, such as job and wealth creation from the automotive industry, transportation provision, societal well-being from leisure and travel opportunities, and revenue generation from the taxes. 


NOUN CANDIDATES:
benefits
benefits
job
wealth
creation
industry
transportation
provision
leisure
travel
opportunities
revenue
generation
taxes
ENTITIES: 
######################################################


SENTENCE:  The ability for people to move flexibly from place to place has far-reaching implications for the nature of societies. 


NOUN CANDIDATES:
ability
people
place
place
implications
nature
societies
ENTITIES: 
######################################################


SENTENCE:  It was estimated in 2014 that the number of cars was over 1. 


NOUN CANDIDATES:
number
cars
ENTITIES: 
2014 DATE
1 CARDINAL
######################################################


SENTENCE:  25 billion vehicles, up from the 500 million of 1986. 


NOUN

'\nUseless entity types:\nCARDINAL,\nDATE\n'

In [9]:
# Testing stuff
for i, doc in enumerate(nlp.pipe(sentences)):
    for ent in doc.ents:
        print("{}\t{}\t{}\t{}".format(ent.text, ent.start, ent.end, ent.label_))

1	14	15	CARDINAL
one to eight	12	15	CARDINAL
four	18	19	CARDINAL
the 20th century	6	9	DATE
The year 1886	0	3	DATE
the birth year	6	9	DATE
German	14	15	NORP
Karl Benz	16	18	PERSON
Benz Patent-Motorwagen	20	24	ORG
the early 20th century	5	9	DATE
One	0	1	CARDINAL
first	3	4	ORDINAL
1908	13	14	DATE
Model T	14	16	PERSON
American	18	19	NORP
the Ford Motor Company	22	26	ORG
US	6	7	GPE
Western Europe	26	28	LOC
nCars	0	1	ORG
the decades	1	3	DATE
the 2010s	5	7	DATE
2008	20	21	DATE
2014	4	5	DATE
1	12	13	CARDINAL
25 billion	0	2	CARDINAL
the 500 million	6	9	CARDINAL
1986	10	11	DATE
China	8	9	GPE
India	10	11	GPE
Latin	13	14	LANGUAGE
vehicle\	20	21	PERSON
English	27	28	LANGUAGE
cart\	35	36	ORG
Old North French	39	42	NORP
Gaulish	7	8	NORP
Gallic	12	13	NORP
1895	6	7	DATE
British	17	18	NORP
English	18	19	LANGUAGE
1895	10	11	DATE
car\	6	7	PERSON
carriage\	3	4	ORG
first	13	14	ORDINAL
1895	28	29	DATE
Greek	12	13	NORP
aut\u00f3s	14	15	PERSON
Latin	25	26	LANGUAGE
\"movable\	30	31	PERSON
English	3	4	LANGUAGE
F

In [18]:
displacy.render(nlp(sentences[6]), style='ent', jupyter=True)

In [11]:
for i, doc in enumerate(nlp.pipe(sentences)):
    for ent in doc.noun_chunks:
        print("{}\t{}\t{}\t{}".format(ent.text, ent.start, ent.end, ent.label_))

batchcomplete":"","warnings":{"extracts":{"*":"\"exlimit\	0	1	NP
a whole article extracts request	6	11	NP
query":{"normalized":[{"from":"car","to":"Car"}],"pages":{"13673345":{"pageid":13673345,"ns":0,"title":"Car","extract":"A car	0	2	NP
automobile	4	5	NP
a wheeled motor vehicle	7	11	NP
transportation	13	14	NP
Most definitions	0	2	NP
car	3	4	NP
they	5	6	NP
roads	9	10	NP
eight people	14	16	NP
four tires	18	20	NP
people	24	25	NP
goods	27	28	NP
Cars	0	1	NP
global use	3	5	NP
the 20th century	6	9	NP
developed economies	11	13	NP
them	15	16	NP
The year	0	2	NP
the birth year	6	9	NP
the modern car	10	13	NP
German inventor Karl Benz	14	18	NP
his Benz Patent-Motorwagen	19	24	NP
Cars	0	1	NP
the early 20th century	5	9	NP
the first cars	2	5	NP
the masses	9	11	NP
the 1908 Model T	12	16	NP
an American car	17	20	NP
the Ford Motor Company	22	26	NP
Cars	0	1	NP
the US	5	7	NP
they	9	10	NP
animal-drawn carriages	11	15	NP
carts	16	17	NP
Western Europe	26	28	NP
other parts	29	31	NP
the world	32	34	NP
nCars	0

its life	44	46	NP
the value	51	53	NP
the benefits	54	56	NP
society	57	58	NP
car use	59	61	NP
The societal benefits	0	3	NP
economy benefits	6	8	NP
job and wealth creation	11	15	NP
car production	17	19	NP
maintenance	20	21	NP
transportation provision	22	24	NP
society	25	26	NP
leisure and travel opportunities	29	33	NP
revenue generation	35	37	NP
the tax opportunities	38	41	NP
The ability	0	2	NP
humans	3	4	NP
place	8	9	NP
place	10	11	NP
far-reaching implications	12	16	NP
the nature	17	19	NP
societies	20	21	NP
n\n\n== Environmental impact	0	3	NP
different types	8	10	NP
fuel	11	12	NP
cars	15	16	NP
gasoline	20	21	NP
diesel	22	23	NP
The United States Environmental Protection Agency	0	6	NP
the average vehicle	8	11	NP
8,887 grams	12	14	NP
the greenhouse gas carbon dioxide	15	20	NP
CO2	21	22	NP
gallon	24	25	NP
gasoline	26	27	NP
The average vehicle	0	3	NP
diesel fuel	5	7	NP
10,180 grams	9	11	NP
carbon dioxide	12	14	NP
Many governments	0	2	NP
fiscal policies	4	6	NP
road tax	9	11	NP
the US gas guzzl

In [19]:
import random

def _choose_nouns(noun_pos):
    amount = len(noun_pos)-1
    first = random.randint(0,amount)
    second = random.randint(0,amount)
    while first == second:
        second = random.randint(0,amount)
    if(first > second):
        tmp = first
        first = second
        second = tmp
    print("Chosen: ",first,second)
    return [noun_pos[first],noun_pos[second]]
    
def _count_consecutive_tags(s,pos):
    count = 0
    for i in range(pos,len(s)):
        if(s[i].tag_ == s[pos].tag_):
            count += 1
        else:
            return count
    return count

def mark_positions(s, ent_pos):
    """
    Mark out e1 and e2 at positions specified in ent_pos
    """
    # chars that we in general dont want a space before
    nospace_chars = [",",".",";",":"]
    return_string = ""
    added = 1
    spacing = ""
    for i in range(len(s)):
        
        if(s[i].text in nospace_chars):
            spacing = ""
            
        if(added < 3 and i == ent_pos[0][0]):
            return_string += " <e1>"
            spacing = ""
        elif(added < 3 and i == ent_pos[0][1]):
            return_string += "</e1>"
            added +=1
        elif(added < 3 and i == ent_pos[1][0]):
            return_string += " <e2>"
            spacing = ""
        elif(added < 3 and i == ent_pos[1][1]):
            return_string += "</e2>" 
            added +=1
        return_string += spacing+s[i].text
        spacing = " "
    return return_string

# Mark up sentences
def mark_sentence(s):
    """
    Marks two nominals with e1 and e2.
    Will select entities that are not CARDINAL or DATE if such exist.
    If not, nouns will be considered instead. Which two nouns get marked is 
    randomized.
    Input: spaCy doc
    Output: string
    """
    return_string = ""
    entities = s.ents
    entity_pos = []
    for e in entities:
        if(e.label_ not in ["CARDINAL","DATE","ORDINAL","TIME","PERCENT","MONEY"]):
            if(entity_pos and entity_pos[-1][1] == e.start):
                continue
            entity_pos.append((e.start,e.end))
    if(len(entity_pos)>= 2):
        return mark_positions(s,entity_pos)
    
    # Not enough entities, mark nouns instead
    noun_pos = entity_pos
    for w in s:
        if(w.tag_ in ["NNS","NN"]):
            if(noun_pos and w.i <= noun_pos[-1][1]):
                continue
            end_pos = _count_consecutive_tags(s,w.i)
            if(end_pos>1):
                print(s[w.i:w.i+end_pos])
            noun_pos.append((w.i,w.i+end_pos))
    print(noun_pos)
    rand_pos = noun_pos
    if(len(noun_pos) > 2):
        # Exchange these lines to swap between adjacent and random selection
        #rand_pos = _choose_nouns(noun_pos)
        rand_pos = noun_pos[:2]
    if(len(noun_pos)>=2):
        return mark_positions(s,rand_pos)
    return ""

In [20]:

#for w in sentences_list[7]:
#    if(w.tag_ in ["NNS","NN"]):
#        print(w,w.i)

#sentences_list[7].ents[0].start
#sentences_list[7].ents[0].end
#sentences_list[7].ents[0].label_ == "GPE"

semeval_texts = []

for s in sentences_list:
    semtext = mark_sentence(s)
    if not semtext:
        continue
    semeval_texts.append(semtext)
    #if(mark_sentence(s) == "xD"):
    #    print(s)
#len(sentences_list[7])
print(semeval_texts)
#mark_positions(sentences_list[7],[(6, 7), (26, 28)])
#sentences_list[28].ents[2].start

[(0, 1), (8, 9), (10, 11)]
motor vehicle
[(1, 2), (4, 5), (9, 11), (13, 14)]
[(1, 2), (3, 4), (9, 10), (15, 16), (19, 20), (24, 25), (27, 28)]
[(0, 1), (4, 5), (8, 9), (12, 13)]
[(0, 1), (8, 9)]
passenger comfort
[(0, 1), (2, 3), (4, 5), (6, 7), (8, 10), (11, 12), (16, 17), (18, 19)]
[(2, 3), (5, 6), (7, 8), (12, 13)]
air conditioning
car entertainment
[(0, 1), (3, 4), (6, 8), (9, 10), (15, 17)]
combustion engine
[(1, 2), (3, 4), (12, 14), (18, 19), (21, 22)]
air pollution
[(2, 4), (9, 10), (12, 13)]
[(0, 1), (3, 4), (6, 7), (9, 10), (13, 14), (18, 19), (21, 22)]
[(1, 2), (9, 10), (12, 13)]
car use
[(2, 3), (4, 5), (6, 8)]
[(1, 2), (5, 6), (7, 8), (12, 13), (17, 18), (19, 20), (21, 22), (23, 24), (26, 27), (28, 29), (31, 32), (34, 35)]
land use
road congestion
air pollution
health care
[(1, 2), (3, 4), (6, 7), (8, 10), (11, 13), (14, 16), (18, 19), (20, 22), (27, 28), (30, 31), (33, 34)]
Road traffic
[(0, 2), (6, 7), (8, 9), (11, 12)]
demand transportation
[(1, 2), (5, 7), (8, 9), (10,

In [21]:
# Write to file
sentence_id = 0

data_file = "test_sentences.txt"
fp = open(data_file,'w')

for s in semeval_texts:
    fp.write(str(sentence_id)+'\t')
    fp.write("\""+s+"\""+"\n")
    #fp.write("\n")
    sentence_id +=1
fp.close()